In [1]:
# conda install fastai


In [1]:
from fastai import *
import fastai
from fastai.text.all import *
# from fastai.text import *
# from fastai.test_utils import *
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [2]:
import torch
torch.cuda.is_available()
# fastai.__version__

True

In [3]:
path = Path('./data')

df = pd.read_csv(path/'final_cleaned_data.csv')

In [4]:
bs = 20

In [5]:
df.head()

Unnamed: 0  \
0           0   
1           1   
2           2   
3           3   
4           4   

                                                                                                                       text  
0                                  Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest   
1                                       Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest   
2                   #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.  
3                  You hate farmers I hate you,  if you love them I love you! ;)   #StopHateAgainstFarmers  #FarmersProtest  
4  They can't be farmers. Looks like Gundas are having good time. They seem to be violence thirsty goons. #FarmersProtest …

In [6]:
df=df.iloc[:, 1:]

In [7]:
df.head()

text
0                                  Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest 
1                                       Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest 
2                   #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.
3                  You hate farmers I hate you,  if you love them I love you! ;)   #StopHateAgainstFarmers  #FarmersProtest
4  They can't be farmers. Looks like Gundas are having good time. They seem to be violence thirsty goons. #FarmersProtest …

In [9]:
# df = df.sample(n=200000, random_state=30)

In [10]:
# !conda install --file requirements.txt


In [8]:
print(fastai.__version__)

2.5.3


In [9]:
valid = np.random.choice([False,True], len(df['text']), p= [0.9, 0.1])
df['is_valid'] = valid
df.head()

text  \
0                                  Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest    
1                                       Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest    
2                   #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.   
3                  You hate farmers I hate you,  if you love them I love you! ;)   #StopHateAgainstFarmers  #FarmersProtest   
4  They can't be farmers. Looks like Gundas are having good time. They seem to be violence thirsty goons. #FarmersProtest …   

   is_valid  
0     False  
1     False  
2     False  
3     False  
4     False

In [10]:
# from lang_model import *
# data_lm = lang_model(df, path, 'text', bs)
data_lm = DataBlock(blocks=TextBlock.from_df('text', is_lm=True),
                    get_x=ColReader('text'),
                    splitter=ColSplitter())
dls = data_lm.dataloaders(df, bs=bs, seq_len=72)
dls.show_batch(max_n=6)


In [11]:
learn = language_model_learner(dls, AWD_LSTM, metrics=accuracy,drop_mult=0.3)

In [14]:

# data_lm = (TextList.from_csv(csv_name='final_cleaned_data.csv', path=path, cols='text')
#            #We may have other temp folders that contain text files so we only keep what's in train and test
#             .split_by_rand_pct(0.1)
#            #We randomly split and keep 10% (10,000 reviews) for validation
#             .label_for_lm()           
#            #We want to do a language model so we label accordingly
#             .databunch(bs=bs))
# data_lm.save('data_lm.pkl')

In [12]:
import gc
# del variables
gc.collect()
torch.cuda.empty_cache()

In [13]:
learn.lr_find()
# learn.recorder.plot(skip_end=15)

SuggestedLRs(valley=0.0020892962347716093)

In [17]:
# lrs,moms = learn.recorder.,learn.recorder.hps['mom']

In [14]:
learn.fit_one_cycle(1, 0.0020892962347716093, moms=(0.8,0.75, 0.7))

In [15]:
learn.save('fit_head')

Path('models/fit_head.pth')

In [16]:
learn.load('fit_head')

In [17]:
learn.unfreeze()

In [18]:
learn.fit_one_cycle(10, 0.00020892962347716093,cbs=[EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=3),SaveModelCallback(monitor='accuracy', min_delta=0.01)], moms=(0.8,0.75,0.7))

Better model found at epoch 0 with accuracy value: 0.38439738750457764.
Better model found at epoch 1 with accuracy value: 0.4096237123012543.
Better model found at epoch 2 with accuracy value: 0.4249083399772644.
Better model found at epoch 4 with accuracy value: 0.4405326843261719.
No improvement since epoch 4: early stopping


In [19]:
learn.save('fine_tuned')

Path('models/fine_tuned.pth')

In [20]:
learn.load('fine_tuned')

In [21]:
learn.predict('covidiots', n_words=40)

'covidiots style of this # BJP # farmersprotest … a water overlooked but despite facing storms , are still penniless , what are their demands prabhu , increased land ache cases hu punjab . # tomarrepealonlysolution # farmersprotest'